Three simple programs to download images of 20 country flags from the web. The first one, flags.py, runs sequentially: it only requests the next image when the previous one is downloaded and saved locally. The other two scripts make concurrent downloads: they request several images practically at the same time, and save them as they arrive. The flags_threadpool.py script uses the concurrent.futures package, while flags_asyncio.py uses asyncio.      https://learning.oreilly.com/library/view/fluent-python-2nd/9781492056348/ch20.html#idm46582389726560

A synchronous Sequential Download Script:


In [1]:
import time
from pathlib import Path
from typing import Callable

import httpx

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'https://www.fluentpython.com/data/flags'
DEST_DIR = Path('downloaded')  

def save_flag(img: bytes, filename: str) -> None: 
    (DEST_DIR / filename).write_bytes(img)

def get_flag(cc: str) -> bytes:
    url = f'{BASE_URL}/{cc}/{cc}.gif'.lower()
    resp = httpx.get(url, timeout=6.1, 
                     follow_redirects=True)
    resp.raise_for_status() 
    return resp.content

def download_many(cc_list: list[str]) -> int: 
    for cc in sorted(cc_list):    
        image = get_flag(cc)
        save_flag(image, f'{cc}.gif')
        print(cc, end=' ', flush=True) 
    return len(cc_list)

def main(downloader: Callable[[list[str]], int]) -> None: 
    DEST_DIR.mkdir(exist_ok=True)  
    t0 = time.perf_counter() 
    count = downloader(POP20_CC)
    elapsed = time.perf_counter() - t0
    print(f'\n{count} downloads in {elapsed:.2f}s')

if __name__ == '__main__':
    main(download_many)

BD BR CD CN DE EG ET FR ID IN IR JP MX NG PH PK RU TR US VN 
20 downloads in 9.81s


Threaded download script using futures.ThreadPoolExecutor

In [ ]:
from concurrent import futures

from flags import save_flag, get_flag, main

def download_one(cc: str):
    image = get_flag(cc)
    save_flag(image, f'{cc}.gif')
    print(cc, end=' ', flush=True)
    return cc

def download_many(cc_list: list[str]) -> int:
    with futures.ThreadPoolExecutor() as executor:  
        res = executor.map(download_one, sorted(cc_list))

    return len(list(res))  

if __name__ == '__main__':
    main(download_many)
